In [1]:
# !conda uninstall pytorch -y

In [2]:
# !pip uninstall torch
# !pip uninstall torch

In [3]:
# !conda config --set ssl_verify no
# !conda install pytorch torchvision torchaudio -c pytorch -y
# !conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 -c pytorch -y

In [4]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [5]:
import torch
torch.__version__

'1.8.0'

In [6]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /Users/ag23268/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [7]:
g = dataset[0]

In [8]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


Defining a Graph Convolutional Network (GCN)
This tutorial will build a two-layer Graph Convolutional Network (GCN). Each layer computes new node representations by aggregating neighbor information.

To build a multi-layer GCN you can simply stack dgl.nn.GraphConv modules, which inherit torch.nn.Module.



In [9]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [10]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.184 (best 0.184), test acc: 0.190 (best 0.190)
In epoch 5, loss: 1.895, val acc: 0.578 (best 0.578), test acc: 0.592 (best 0.592)
In epoch 10, loss: 1.817, val acc: 0.550 (best 0.578), test acc: 0.562 (best 0.592)
In epoch 15, loss: 1.716, val acc: 0.606 (best 0.606), test acc: 0.625 (best 0.625)
In epoch 20, loss: 1.594, val acc: 0.666 (best 0.666), test acc: 0.694 (best 0.694)
In epoch 25, loss: 1.454, val acc: 0.676 (best 0.682), test acc: 0.714 (best 0.705)
In epoch 30, loss: 1.300, val acc: 0.714 (best 0.714), test acc: 0.728 (best 0.728)
In epoch 35, loss: 1.140, val acc: 0.726 (best 0.726), test acc: 0.743 (best 0.743)
In epoch 40, loss: 0.980, val acc: 0.738 (best 0.740), test acc: 0.748 (best 0.748)
In epoch 45, loss: 0.828, val acc: 0.744 (best 0.744), test acc: 0.757 (best 0.755)
In epoch 50, loss: 0.690, val acc: 0.754 (best 0.754), test acc: 0.763 (best 0.763)
In epoch 55, loss: 0.568, val acc: 0.760 (best 0.760), test acc: 0.772 (best 0